In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
train_df = pd.read_csv ('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')

In [ ]:
train_df.info()

In [ ]:
train_df.columns

In [ ]:
# 查看缺失值及其比例
per_na = pd.concat([pd.Series(train_df.isna().sum().values),pd.Series((train_df.isna().sum().values / train_df.shape[0])*100)],axis=1)
per_na.columns = ['Na_Counts','percentOfNa']
per_na.index = train_df.columns
per_na

In [ ]:
train_df.describe()

In [ ]:
# 填充缺失值，因为Cabin缺失值比例>20,所以直接删除不考虑
# 而Embarked是类别标签，缺失值较少，所以可以用出现最多的标签去填充
train_data = train_df.copy()#创建副本
train_data = train_data.drop('Cabin',axis=1)
train_data['Embarked'].fillna(max(dict(train_data['Embarked'].value_counts())),inplace=True) 

In [ ]:
train_data.isna().sum()

In [ ]:
#查看Age分布，看看有无左右偏的情况
ax = train_data['Age'].hist(bins=15,density=True,stacked=True,color='teal',alpha=0.6)
train_data['Age'].plot(kind='density',color='teal')
ax.set(xlabel='Age')
plt.xlim(-10,85)
plt.show()

In [ ]:
# 有点右偏，所以选择中位数填充好一些
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median(skipna=True))

In [ ]:
# 删除
train_data = train_data.drop('Name',axis=1)
train_data = train_data.drop('Ticket',axis=1)
train_data = train_data.drop('PassengerId',axis=1)

In [ ]:
# 查看缺失值及其比例
per_na = pd.concat([pd.Series(train_data.isna().sum().values),pd.Series((train_data.isna().sum().values / train_data.shape[0])*100)],axis=1)
per_na.columns = ['Na_Counts','percentOfNa']
per_na.index = train_data.columns
per_na


# 处理测试集

In [ ]:
test_df = test_df.drop('Name',axis=1)
test_df = test_df.drop('Ticket',axis=1)

In [ ]:
# 对测试集来进行处理
# 查看缺失值及其比例
per_na = pd.concat([pd.Series(test_df.isna().sum().values),pd.Series((test_df.isna().sum().values / test_df.shape[0])*100)],axis=1)
per_na.columns = ['Na_Counts','percentOfNa']
per_na.index = test_df.columns
per_na

In [ ]:
# 同样处理
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].median(skipna=True))
test_df['Fare'] = test_df['Fare'].fillna(test_df['Fare'].median(skipna=True))
test_df = test_df.drop('Cabin',axis=1)
per_na = pd.concat([pd.Series(test_df.isna().sum().values),pd.Series((test_df.isna().sum().values / test_df.shape[0])*100)],axis=1)
per_na.columns = ['Na_Counts','percentOfNa']
per_na.index = test_df.columns
per_na

# 查看特征

In [ ]:
# 查看特征
#填充效果可视化
plt.figure(figsize=(15,8))
ax = train_df['Age'].hist(bins=15,density=True,stacked=True,color='teal',alpha=0.6)
train_df['Age'].plot(kind='density',color='teal')
ax = train_data['Age'].hist(bins=15,density=True,stacked=True,color='orange',alpha=0.5)
train_data['Age'].plot(kind='density',color='orange')
ax.set(xlabel='Age')
plt.xlim(-10,85)
plt.legend(['Ordinary Age','Adjusted Age'])
plt.show()

In [ ]:
# 异常值处理
#box plot overallqual/saleprice
list1 = []
for i in train_data.columns.tolist()[1:]:
    if i not in train_data.columns[train_data.dtypes =='object'].tolist():
        list1.append(i)
plt.figure(figsize=(25,15))
for i in range(len(list1)):
    ax = plt.subplot(231+i)
    sns.boxplot(x='Survived',y=list1[i],data=train_data)

In [ ]:
sns.barplot('Pclass',y='Survived',data=train_data,color="darkturquoise")
plt.legend(['Dead','Survived'])
plt.show()

In [ ]:
sns.barplot('Embarked', 'Survived', data=train_data, color="teal")
plt.show()

In [ ]:
sns.barplot('Sex', 'Survived', data=train_df, color="aquamarine")
plt.show()

In [ ]:
train_data.columns

In [ ]:
train_data.dtypes

In [ ]:
from sklearn import preprocessing
f_names = ['Sex','Embarked']
train_data_Enlabel = train_data.copy()
for x in f_names:
    label = preprocessing.LabelEncoder()
    train_data_Enlabel[x] = label.fit_transform(train_data_Enlabel[x])
feature_col = train_data_Enlabel.columns
feature_col = feature_col.tolist()
sns.pairplot(train_data_Enlabel[feature_col],hue="Survived")

In [ ]:
# 独热编码化
label_list = list(dict(train_data.dtypes[train_data.dtypes =='object']).keys())
train_data = pd.concat([train_data,pd.get_dummies(train_data[label_list])],axis=1)
train_data = train_data.drop(label_list,axis=1)

In [ ]:
cmap = sns.diverging_palette(220, 10, as_cmap=True) # 设置配色
ax=sns.clustermap(train_data.corr(),cmap=cmap,figsize=(8,8),annot=True)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca = PCA(n_components = 1)
x = train_data[['SibSp', 'Parch']]
x = StandardScaler().fit_transform(x)
principalComponents = pca.fit_transform(x)
train_data['PCA_sibspAndParch'] = principalComponents

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
# pip install -U scikit-learn
#要升级kaggle里面的sklearn版本，不然会报错，
# kaggle环境版本太低了，所以直接在下面写了输出结果


# from sklearn.feature_selection import RFECV
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import RFE
# # Create the RFE object and compute a cross-validated score.
# # The "accuracy" scoring is proportional to the number of correct classifications
# X = train_data[train_data.columns[1:]]
# X = X.astype('float64')
# y = train_data['Survived']
# rfecv = RFECV(estimator=LogisticRegression(), step=1, cv=5, scoring='accuracy')
# rfecv.fit(X, y)

# print("Optimal number of features: %d" % rfecv.n_features_)
# print('Selected features: %s' % list(X.columns[rfecv.support_]))

# # Plot number of features VS. cross-validation scores
# plt.figure(figsize=(10,6))
# plt.xlabel("Number of features selected")
# plt.ylabel("Cross validation score (nb of correct classifications)")
# plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
# plt.show()


#输出的结果是：
# Optimal number of features: 9
#Selected features: ['Pclass', 'Age', 'SibSp', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'PCA_sibspAndParch']

In [ ]:
Selected_features = ['Pclass', 'SibSp', 'Parch', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'PCA_sibspAndParch']
X = train_data[Selected_features]

plt.subplots(figsize=(8, 5))
sns.heatmap(X.corr(), annot=True, cmap="RdYlGn")
plt.show()
# sibSp 和Parch相关系数比较高，删掉1个或降维

In [ ]:
fe_list=['Pclass', 'Age', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S','PCA_sibspAndParch']

In [ ]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import RepeatedStratifiedKFold
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 
# from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss
# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import LogisticRegression

# X = train_data[fe_list]
# X = StandardScaler().fit_transform(X)
# y = train_data['Survived']

# #Define simple model
# ###############################################################################
# C = np.arange(1e-05, 5.5, 0.1)
# scoring = {'Accuracy': 'accuracy', 'AUC': 'roc_auc', 'Log_loss': 'neg_log_loss'}
# log_reg = LogisticRegression()

# #Simple pre-processing estimators
# ###############################################################################
# std_scale = StandardScaler(with_mean=False, with_std=False)
# #std_scale = StandardScaler()

# #Defining the CV method: Using the Repeated Stratified K Fold
# ###############################################################################

# n_folds=5
# n_repeats=5

# rskfold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=n_repeats, random_state=2022)

# #Creating simple pipeline and defining the gridsearch
# ###############################################################################

# log_clf_pipe = Pipeline(steps=[('scale',std_scale), ('clf',log_reg)])

# log_clf = GridSearchCV(estimator=log_clf_pipe, cv=rskfold,
#               scoring=scoring, return_train_score=True,
#               param_grid=dict(clf__C=C), refit='Accuracy')

# log_clf.fit(X, y)
# results = log_clf.cv_results_

# print('='*20)
# print("best params: " + str(log_clf.best_estimator_))
# print("best params: " + str(log_clf.best_params_))
# print('best score:', log_clf.best_score_)
# print('='*20)

# plt.figure(figsize=(10, 10))
# plt.title("GridSearchCV evaluating using multiple scorers simultaneously",fontsize=16)

# plt.xlabel("Inverse of regularization strength: C")
# plt.ylabel("Score")
# plt.grid()

# ax = plt.axes()
# ax.set_xlim(0, C.max()) 
# ax.set_ylim(0.35, 0.95)

# X_axis = np.array(results['param_clf__C'].data, dtype=float)

# for scorer, color in zip(list(scoring.keys()), ['g', 'k', 'b']): 
#     for sample, style in (('train', '--'), ('test', '-')):
#         sample_score_mean = -results['mean_%s_%s' % (sample, scorer)] if scoring[scorer]=='neg_log_loss' else results['mean_%s_%s' % (sample, scorer)]
#         sample_score_std = results['std_%s_%s' % (sample, scorer)]
#         ax.fill_between(X_axis, sample_score_mean - sample_score_std,
#                         sample_score_mean + sample_score_std,
#                         alpha=0.1 if sample == 'test' else 0, color=color)
#         ax.plot(X_axis, sample_score_mean, style, color=color,
#                 alpha=1 if sample == 'test' else 0.7,
#                 label="%s (%s)" % (scorer, sample))

#     best_index = np.nonzero(results['rank_test_%s' % scorer] == 1)[0][0]
#     best_score = -results['mean_test_%s' % scorer][best_index] if scoring[scorer]=='neg_log_loss' else results['mean_test_%s' % scorer][best_index]
        
#     # Plot a dotted vertical line at the best score for that scorer marked by x
#     ax.plot([X_axis[best_index], ] * 2, [0, best_score],
#             linestyle='-.', color=color, marker='x', markeredgewidth=3, ms=8)

#     # Annotate the best score for that scorer
#     ax.annotate("%0.2f" % best_score,
#                 (X_axis[best_index], best_score + 0.005))

# plt.legend(loc="best")
# plt.grid('off')
# plt.show()

In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# pca = PCA(n_components = 1)
# x = test_df[['SibSp', 'Parch']]
# x = StandardScaler().fit_transform(x)
# principalComponents = pca.fit_transform(x)
# test_df['PCA_sibspAndParch'] = principalComponents
# test_df = pd.concat([test_df,pd.get_dummies(test_df[['Sex','Embarked']])],axis=1)
# final_test=pd.DataFrame()
# final_test['PassengerId'] = test_df['PassengerId']
# final_test['Survived'] = log_clf.predict(StandardScaler().fit_transform(test_df[fe_list]))


# submission = final_test[['PassengerId','Survived']]

# submission.to_csv("submission.csv", index=False)

# submission.tail()


In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(train_data[train_data.columns[1:]], train_data[train_data.columns[0]], test_size=0.3, random_state=42)

# 随机森林调参

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# rfc=RandomForestClassifier(random_state=2022)

In [ ]:
# param_grid = { 
#     'n_estimators': list(range(1,11)),
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [4,5,6,7,8],
#     'criterion' :['gini', 'entropy']
# }

In [ ]:
# CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
# CV_rfc.fit(x_train, y_train)

In [ ]:
# CV_rfc.best_params_

In [ ]:
# rfc1=RandomForestClassifier(random_state=2022, criterion='gini',max_depth=4,max_features='auto',n_estimators= 7)

In [ ]:
# rfc1.fit(x_train, y_train)
# pred=rfc1.predict(x_test)
# print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred))

In [ ]:
# op_rf=rfc1.predict(test_df.drop(['Sex','Embarked'],axis=1).iloc[:,1:][x_train.columns])

In [ ]:
# op=pd.DataFrame(test_df['PassengerId'])
# op['Survived']=op_rf
# op.to_csv("op_rf.csv", index=False)

# 模型融合

In [ ]:
# op_rf=rfc1.predict_proba(test_df.drop(['Sex','Embarked'],axis=1).iloc[:,1:][x_train.columns])
# log_clf1 = log_clf.predict_proba(StandardScaler().fit_transform(test_df[fe_list]))

In [ ]:
# df_result = pd.DataFrame(op_rf+log_clf1) #直接把两个模型预测的概率相加

In [ ]:
# df_result

In [ ]:
# ave_result=pd.DataFrame()
# ave_result['PassengerId'] =test_df['PassengerId']
# ave_result['Survived'] = df_result.idxmax(1) # 取出概率的分类类别

In [ ]:
# ave_result.to_csv('./ave_result.csv',index=False)

# XGB

In [ ]:
final_test=pd.DataFrame()
final_test['PassengerId'] = test_df['PassengerId']

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# 载入必要的库
import pandas as pd
import numpy as np
import xgboost as xgb

import missingno as msno
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
pca = PCA(n_components = 1)
x = test_df[['SibSp', 'Parch']]
x = StandardScaler().fit_transform(x)
principalComponents = pca.fit_transform(x)
test_df['PCA_sibspAndParch'] = principalComponents
test_df = pd.concat([test_df,pd.get_dummies(test_df[['Sex','Embarked']])],axis=1)

In [ ]:
test = test_df
features = fe_list
train = train_data
train[features]

In [ ]:
# print(DTR.get_params())
params = {"objective": "reg:logistic", # for linear regression
          "eta": 0.004,   # learning rate
          "max_depth": 12,    # maximum depth of a tree
          "subsample": 0.7,    # Subsample ratio of the training instances
          "colsample_bytree": 0.5,   # Subsample ratio of columns when constructing each tree
          "silent": 1,   # silent mode
          "seed": 10   # Random number seed
          }
num_trees = 9500

In [ ]:
X_train, X_test = train_test_split(train, test_size=0.2, random_state=2022)

dtrain = xgb.DMatrix(X_train[features], np.log1p(X_train.Survived))
dvalid = xgb.DMatrix(X_test[features], np.log1p(X_test.Survived))
dtest = xgb.DMatrix(test[features])

watchlist = [(dtrain, 'train'),(dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, early_stopping_rounds=50, verbose_eval=False)

In [ ]:
test_probs = gbm.predict(xgb.DMatrix(test[features]), ntree_limit=gbm.best_ntree_limit)
indices = test_probs < 0
indices2 = test_probs > 0.5
indices3 = test_probs <= 0.5
test_probs[indices] = 0
test_probs[indices2] = 1
test_probs[indices3] = 0
test_probs =test_probs.astype('int')
submission = pd.DataFrame({"PassengerId": final_test["PassengerId"], "Survived": test_probs})
submission.to_csv("./XGB.csv", index=False)